In [ ]:
# import cProfile
# import collections
import datetime

import os
import pickle
# import sys
import time

# import numpy as np
import pandas as pd

# from datetime import datetime
from dotenv import load_dotenv

pd.options.display.float_format = "{:.2f}".format
# plt.rcParams.update({'font.size': 18})

In [ ]:
load_dotenv()
SPOTIPY_API = os.getenv('SPOTIPY_API')
SPOTIPY_API_KEY = os.getenv('SPOTIPY_API_KEY')
scope=''

# https://spotipy.readthedocs.io/en/latest/#
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIPY_API, client_secret=SPOTIPY_API_KEY)

# help(sp)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager,requests_timeout=300)

# 1. Get user and playlist IDs

In [ ]:
n = 950 # max = 950
p_limit = 50 # limit of playlists max = 50
years = range(2018,2022,1)

In [ ]:
# Extract the user and playlist IDs
format = '%d/%m/%Y %H:%M'
t = time.time()

# check if data extraction already exists then load if it exists, otherwise performs the data request from API and save the file
data_path = os.getcwd() + '\\data\\external\\raw_1_uid_pid.pkl'
if os.path.exists(data_path):
    print('Playlist ID and User ID - Data already requested from API - Loaded: ' + data_path)
    # load file only if the next step was not executed (getting tracks)
    data_path_2 = os.getcwd() + '\\data\\external\\raw_2_playlists.pkl'
    if not(os.path.exists(data_path_2)):
        with open(data_path, 'rb') as f:
            uid_pid_raw = pickle.load(f)
else:
    print('Playlist ID and User ID - Data requested from API - Saved: ' + data_path)
    # list of all user ids (UID) and playlist ids (PID)
    uid_pid_raw = []
    
    for y in years:
        txt = 'year:' + str(y)
        print(txt)
        for i in range(0,n,1):
            if i % 100 == 0:
                t_now = datetime.datetime.now().strftime(format)
                calc_eta = datetime.datetime.now() + datetime.timedelta(seconds = n - i) * (time.time() - t) / (i + 1)
                calc_eta = calc_eta.strftime(format)
                print("%s - %d/%d elapsed time (min): %.1f - ETA: %s" % (t_now, i, n, (time.time()-t)/60 ,calc_eta))
            uid_pid = sp.search(q=txt, type='playlist', limit=p_limit, offset=i,market='US')
            uid_pid_raw.append(uid_pid)
    # Save the data
    with open(data_path, 'wb') as f:
        pickle.dump(uid_pid_raw, f)
        
print(len(uid_pid_raw)) # CHECK WHY THE COUNT OF PLAYLISTS DECREASED

# 2. Get the details of each playlist

In [ ]:
# extract = True
extract = False

# Get the detail of all playlists
format = '%d/%m/%Y %H:%M'
t = time.time()

n = len(uid_pid_raw)
data_path = os.getcwd() + '\\data\\external\\raw_2_playlists_N.pkl'
if extract == True:
    print('Details of each playlist - Data requested from API - Saved:' + data_path)
    # Get details of all playlists
    playlists_full = []
    print('Getting details of all playlists')
    for i, uid_pid in enumerate(uid_pid_raw):
        if i % 100 == 0 and i > 0:
            playlists_full = []
            t_now = datetime.datetime.now().strftime(format)
            calc_eta = datetime.datetime.now() + datetime.timedelta(seconds = n - i) * (time.time() - t) / (i + 1)
            calc_eta = calc_eta.strftime(format)
            print("%s - %d/%d elapsed time (min): %.1f - ETA: %s" % (t_now, i, n, (time.time()-t)/60 ,calc_eta))

        for user in uid_pid['playlists']['items']:
            pid = user['id']
            uid = user['owner']['id']
            playlist = sp.user_playlist(uid, pid)
            playlists_full.append(playlist)

        # Save the data
        if i % 100 == 0 and i > 0:
            print(i)
            data_path = os.getcwd() + '\\data\\external\\raw_2_playlists_' + str(int(i/100)) + '.pkl'
            print(data_path)
            with open(data_path, 'wb') as f:
                pickle.dump(playlists_full, f)